In [1]:
from jettool import tools
import pandas
import numpy

# 初始化

In [2]:
# 初始化物件，可同時set Key
tejtool = tools.financial_tool(api_key="GDEy0mWAGqnI3EemCREGREZMcEVbnF")
# tejtool.load_data()
# 設定keyID，會自動查詢需要的資料表索引構造
#tejtool.set_apikey("GDEy0mWAGqnI3EemCREGREZMcEVbnF")

['使用者名稱：趙育祥(ZYX)', '使用權限日期：2018-01-01/2050-12-31', '日連線次數狀態：657/100000', '日查詢資料量狀態：3698459/10000000', '月查 詢資料量狀態：47693080/9223372036854775807']


# 資料結構資訊

In [ ]:
# 尋找某個table的mapping資料次屬性，可以用來知道你查的table，在其他國家是什麼table
tejtool.get_table_mapping(market='TWN',id='AIND')

In [ ]:
# 取得該table除了PK以外的欄位名稱，用來簡化查詢步驟，無腦一口氣整個表匯入
tejtool.get_table_columns('TWN/AFF_RAW')

In [ ]:
# 用模糊字串來查詢確切的會計科目名稱
tejtool.find_account_name('資產負債')

# 整合查詢器
### 一次查詢全部整合完成
#### 一個名稱、可到不同資料表查詢相對應的資料
#### 需正確定義名稱的精確差異

#### unfixed bug 

pipeline\method.py in cal_zdate(self, base_date, jump_length, jump_kind, fix_date, tradeday)   
ValueError: Month out of range in datetime string "2018-22-31"  
data = tejtool.query_data(base_date='2019-12-31',window='14m',column_names=data_name)

In [3]:
#data_name = ['常續性稅後淨利','收盤價(元)','報酬率-Ln','個別風險','長期反轉(T-60~T-13)','董監持股%','大股東持股(TSE)%','經理人持股%','單月營收(千元)']
fama_name = tejtool.get_table_columns('TWN/AFF_RAW')
data_name = ['常續性稅後淨利','收盤價(元)','報酬率-Ln','單月營收(千元)']
data_name = data_name + fama_name
data = tejtool.query_data(base_date='2019-12-31',window='12m',column_names=data_name)
tejtool.save_data()
data[data['coid']=='2330']

資料起始日：2018-12-31
查詢財報資料
成功查詢會計家數:1970
最大財報資料日期:2019-12-01
查詢日資料 最大資料日期:2019-12-31
上市(櫃)月營收盈餘 重新查詢:True
上市(櫃)調整股價(日)-除權息調整 重新查詢:True
台灣_多因子DB 重新查詢:True
借券餘額 重新查詢:True


,zdate,mdate,coid,投資(資產成長率),產業集中度,股利殖利率,研發密度,動能(T-12~T-2),淨值市價比,市值,...,流動性,單月營收(千元),錯誤定價,違約距離(KMV),違約距離(MERTON),個別風險,益本比,收盤價(元),盈利能力(ROE),偏態係數
144838,2019-01-03,2018-12-01,2330,0.21,0.26,0.0,3.4251,0.0,1.18,7.898290e+06,...,1.8458,1299155.0,-43.1924,0.0,0.0,1.9463,-58.43,215.5,-9.48,0.9457
144839,2019-01-04,2018-12-01,2330,0.21,0.26,0.0,3.4251,0.0,1.18,7.898290e+06,...,1.8458,1299155.0,-43.1924,0.0,0.0,1.9463,-58.43,208.0,-9.48,0.9457
144840,2019-01-07,2018-12-01,2330,0.21,0.26,0.0,3.4251,0.0,1.18,7.898290e+06,...,1.8458,1299155.0,-43.1924,0.0,0.0,1.9463,-58.43,213.0,-9.48,0.9457
144841,2019-01-08,2018-12-01,2330,0.21,0.26,0.0,3.4251,0.0,1.18,7.898290e+06,...,1.8458,1299155.0,-43.1924,0.0,0.0,1.9463,-58.43,211.0,-9.48,0.9457
144842,2019-01-09,2018-12-01,2330,0.21,0.26,0.0,3.4251,0.0,1.18,7.898290e+06,...,1.8458,1299155.0,-43.1924,0.0,0.0,1.9463,-58.43,215.5,-9.48,0.9457
144843,2019-01-10,2018-12-01,2330,0.21,0.26,0.0,3.4251,0.0,1.18,7.898290e+06,...,1.8458,89830598.0,-43.1924,0.0,0.0,1.9463,-58.43,216.0,-9.48,0.9457
144844,2019-01-11,2018-12-01,2330,0.21,0.26,0.0,3.4251,0.0,1.18,7.898290e+06,...,1.8458,89830598.0,-43.1924,0.0,0.0,1.9463,-58.43,220.5,-9.48,0.9457
144845,2019-01-14,2018-12-01,2330,0.21,0.26,0.0,3.4251,0.0,1.18,7.898290e+06,...,1.8458,89830598.0,-43.1924,0.0,0.0,1.9463,-58.43,218.5,-9.48,0.9457
144846,2019-01-15,2018-12-01,2330,0.21,0.26,0.0,3.4251,0.0,1.18,7.898290e+06,...,1.8458,89830598.0,-43.1924,0.0,0.0,1.9463,-58.43,221.0,-9.48,0.9457
144847,2019-01-16,2018-12-01,2330,0.21,0.26,0.0,3.4251,0.0,1.18,7.898290e+06,...,1.8458,89830598.0,-43.1924,0.0,0.0,1.9463,-58.43,217.5,-9.48,0.9457


In [4]:
tejtool.get_zdate('2019-08-16')

2019-08-15


[numpy.datetime64('2019-08-15'), numpy.datetime64('2019-08-16')]

In [5]:
tejtool.get_data(base_date='2019-08-16',window='3m',column_names=data_name)

2019-08-15
[numpy.datetime64('2019-08-15'), numpy.datetime64('2019-08-16')]


,zdate,mdate,coid,投資(資產成長率),產業集中度,股利殖利率,研發密度,動能(T-12~T-2),淨值市價比,市值,...,流動性,單月營收(千元),錯誤定價,違約距離(KMV),違約距離(MERTON),個別風險,益本比,收盤價(元),盈利能力(ROE),偏態係數
0,2019-05-16,2019-03-01,9958,4.49,0.05,0.0,0.0000,0.0,1.7,11782933.0,...,0.7729,128912.0,-73.8957,0.0,0.0,2.1068,66.68,60.00,-1.42,0.0816
1,2019-05-17,2019-03-01,9958,4.49,0.05,0.0,0.0000,0.0,1.7,11782933.0,...,0.7729,128912.0,-73.8957,0.0,0.0,2.1068,66.68,60.80,-1.42,0.0816
2,2019-05-20,2019-03-01,9958,4.49,0.05,0.0,0.0000,0.0,1.7,11782933.0,...,0.7729,128912.0,-73.8957,0.0,0.0,2.1068,66.68,64.40,-1.42,0.0816
3,2019-05-21,2019-03-01,9958,4.49,0.05,0.0,0.0000,0.0,1.7,11782933.0,...,0.7729,128912.0,-73.8957,0.0,0.0,2.1068,66.68,63.10,-1.42,0.0816
4,2019-05-22,2019-03-01,9958,4.49,0.05,0.0,0.0000,0.0,1.7,11782933.0,...,0.7729,128912.0,-73.8957,0.0,0.0,2.1068,66.68,62.40,-1.42,0.0816
5,2019-05-23,2019-03-01,9958,4.49,0.05,0.0,0.0000,0.0,1.7,11782933.0,...,0.7729,128912.0,-73.8957,0.0,0.0,2.1068,66.68,63.10,-1.42,0.0816
6,2019-05-24,2019-03-01,9958,4.49,0.05,0.0,0.0000,0.0,1.7,11782933.0,...,0.7729,128912.0,-73.8957,0.0,0.0,2.1068,66.68,62.30,-1.42,0.0816
7,2019-05-27,2019-03-01,9958,4.49,0.05,0.0,0.0000,0.0,1.7,11782933.0,...,0.7729,128912.0,-73.8957,0.0,0.0,2.1068,66.68,62.20,-1.42,0.0816
8,2019-05-28,2019-03-01,9958,4.49,0.05,0.0,0.0000,0.0,1.7,11782933.0,...,0.7729,128912.0,-73.8957,0.0,0.0,2.1068,66.68,62.20,-1.42,0.0816
9,2019-05-29,2019-03-01,9958,4.49,0.05,0.0,0.0000,0.0,1.7,11782933.0,...,0.7729,128912.0,-73.8957,0.0,0.0,2.1068,66.68,62.10,-1.42,0.0816


In [ ]:
tejtool.make_famamacbethmodel(col_name='報酬率-Ln',check_index=['市值', '淨值市價比', '益本比', '股利殖利率', '動能(T-12~T-2)', '短期反轉(T-1)', '長期反轉(T-60~T-13)', '盈利能力(ROE)', '投資(資產成長率)', '違約距離(MERTON)', '違約距離(KMV)', '流動性', '個別風險', '產業集中度', '錯誤定價', '偏態係數', '研發密度'],window ='12m',alpha_rate = 5)

# 回測工具，與quantjet網站同一套

In [ ]:
# 定義計算式，然後使用back_test函式即可回測，目前欠缺自動查詢函式內需要資料欄位(即quantjet網站js檔的功能)的程序

def calculate():
    tejtool.data['EP'] = tejtool.data['常續性稅後淨利']/tejtool.data['收盤價(元)']
    tejtool.data['常續性稅後淨利檢查'] = tejtool.check_above('EP',0,window='1d')
def evaluate():
    tejtool.data['購入'] = tejtool.check_condition(conditions=['常續性稅後淨利檢查'])
    tejtool.data['unit'] = tejtool.equal_pv()

# 另外必須輸入用來當作市價與報酬率的欄位名稱，如:roib_name='報酬率-Ln',closed_name ='收盤價(元)'
# 若不輸入，則使用此預設值
tejtool.back_test(back_interval='2019-10-31',cash=1000000000,calculate=calculate,evaluate=evaluate)

In [ ]:
def calculate(tejtool):
    tejtool.data['報酬率']=tejtool.calculate_growthrate(check_index='收盤價(元)',window='1m',fix_date='01',method='geometric')
    tejtool.data['famamodel']=tejtool.make_famamacbethmodel(col_name='報酬率-Ln',check_index=['市值', '淨值市價比', '益本比', '股利殖利率', '動能(T-12~T-2)', '短期反轉(T-1)', '長期反轉(T-60~T-13)', '盈利能力(ROE)', '投資(資產成長率)', '違約距離(MERTON)', '違約距離(KMV)', '流動性', '個別風險', '產業集中度', '錯誤定價', '偏態係數', '研發密度'],window ='24m',alpha_rate = 5,reset_list = '01')
    tejtool.data['famascore']=tejtool.run_famascore(rank_above=100,class_count = 10) 
    tejtool.data['fama分組當月報酬']=tejtool.calculate_crossing(check_index='報酬率',window='1d',weight='市值',col_kind='mean')
def evaluate(tejtool):
    tejtool.data['fama分組判斷'] =tejtool.abnormal_selection(check_index='fama分組當月報酬',group_name='famascore',window='12m',keep='first')
    tejtool.data['購入']=tejtool.check_condition(conditions=['fama分組判斷'],check_type='and') 
    tejtool.data['unit'] = tejtool.equal_pv()
tejtool.back_test(back_interval='2018-12-31',cash=1000000000,calculate=calculate,evaluate=evaluate)

In [ ]:
tejtool.data.loc[tejtool.data['coid']=='1603',['unit','roibNext','前期持股','收盤價(元)','報酬率-Ln','手續費']]

In [ ]:
tejtool.all_date_data[(tejtool.all_date_data['coid']=='1603')&(tejtool.all_date_data['zdate']>='2019-08-15')]